In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [89]:
summoner_name = "쪼렙이다말로하자"

match = pd.read_json("data/data_summoner_match_data_{name}.json".format(name=summoner_name))
timeline = pd.read_json("data/data_summoner_match_timeline_data_{name}.json".format(name=summoner_name))

# 경기 관련 데이터

In [90]:
match_length = len(match["info"])

In [91]:
info = match["info"]

participants = []
teams = []
for i in range(match_length):
    participants.append(info[i]["participants"])
    teams.append(info[i]["teams"])
    del(info[i]["participants"])
    del(info[i]["teams"])

In [92]:
df_info = pd.json_normalize(info)
df_info_1 = df_info[["gameDuration", "gameMode", "gameType", "gameVersion", "mapId"]]
display(df_info_1.head(10))

,gameDuration,gameMode,gameType,gameVersion,mapId
0,1869582,CLASSIC,MATCHED_GAME,11.19.398.9466,11
1,1266259,ARAM,MATCHED_GAME,11.18.395.7538,12
2,1188428,ARAM,MATCHED_GAME,11.18.395.7538,12
3,1160458,ARAM,MATCHED_GAME,11.18.395.7538,12
4,939280,ARAM,MATCHED_GAME,11.18.395.7538,12
5,1067782,ARAM,MATCHED_GAME,11.18.395.7538,12
6,1160782,ARAM,MATCHED_GAME,11.18.395.7538,12
7,825319,ARAM,MATCHED_GAME,11.18.395.7538,12
8,1162655,ARAM,MATCHED_GAME,11.18.395.7538,12
9,1397866,ARAM,MATCHED_GAME,11.18.395.7538,12


In [93]:
df_participants = pd.json_normalize(participants)
display(df_participants.head(1))

,0,1,2,3,4,5,6,7,8,9
0,"{'assists': 13, 'baronKills': 0, 'bountyLevel'...","{'assists': 16, 'baronKills': 0, 'bountyLevel'...","{'assists': 10, 'baronKills': 1, 'bountyLevel'...","{'assists': 13, 'baronKills': 0, 'bountyLevel'...","{'assists': 26, 'baronKills': 0, 'bountyLevel'...","{'assists': 5, 'baronKills': 0, 'bountyLevel':...","{'assists': 6, 'baronKills': 0, 'bountyLevel':...","{'assists': 14, 'baronKills': 0, 'bountyLevel'...","{'assists': 6, 'baronKills': 1, 'bountyLevel':...","{'assists': 5, 'baronKills': 0, 'bountyLevel':..."


In [94]:
df_teams = pd.json_normalize(teams)
df_teams.columns = ["100", "200"]   # teamId
display(df_teams.head(5))

,100,200
0,"{'bans': [], 'teamId': 100, 'win': True, 'obje...","{'bans': [], 'teamId': 200, 'win': False, 'obj..."
1,"{'bans': [], 'teamId': 100, 'win': True, 'obje...","{'bans': [], 'teamId': 200, 'win': False, 'obj..."
2,"{'bans': [], 'teamId': 100, 'win': False, 'obj...","{'bans': [], 'teamId': 200, 'win': True, 'obje..."
3,"{'bans': [], 'teamId': 100, 'win': False, 'obj...","{'bans': [], 'teamId': 200, 'win': True, 'obje..."
4,"{'bans': [], 'teamId': 100, 'win': False, 'obj...","{'bans': [], 'teamId': 200, 'win': True, 'obje..."


In [95]:
df_teams["100"][0]["win"]

True

In [97]:
df_info_1.to_csv("data/{}/info.csv".format(summoner_name))
df_participants.to_csv("data/{}/participants.csv".format(summoner_name))
df_teams.to_csv("data/{}/teams.csv".format(summoner_name))

# 인베이드 관련 데이터

In [98]:
match_invade = 0

## 이벤트 분석

In [100]:
is_invade = [False]*match_length
invade_timestamp = [0]*match_length           
invade_kill_blue = [0]*match_length           # invade시 발생한 블루팀의 킬 수
invade_kill_red = [0]*match_length            # invade시 발생한 레드팀의 킬 수
invade_win_team = [0]*match_length            # invade가 발생한 게임에서 이긴 팀의 teamId
total_gold_blue_3_min = [0]*match_length
total_gold_red_3_min = [0]*match_length
gold_gap = [0]*match_length

for match_index in range(match_length):
    match_time_length = len(timeline.iloc[match_index].info['frames'])
    
    events = [timeline.iloc[match_index].info["frames"][t]["events"] for t in range(match_time_length)]
    participants_info = [timeline.iloc[match_index].info["frames"][t]["participantFrames"] for t in range(match_time_length)]

    # 1~2분대의 kill event
    if len(events) < 3: 
        print("다시하기 발생!")
        continue    # 게임이 3분 이내로 끝났을 경우를 제외 
    for t in range(1,3):  # 1 ~ 2 분 사이
        for e in events[t]:    
            if e["type"] == "CHAMPION_KILL":
                if is_invade[match_index] == False: 
                    is_invade[match_index] = True
                    match_invade += 1
                if invade_timestamp[match_index] == 0: invade_timestamp[match_index] = e['timestamp']
                if e['killerId'] <= 5:
                    invade_kill_blue[match_index] += 1
                elif e['killerId'] > 5:
                    invade_kill_red[match_index] += 1

    if is_invade[match_index]:
        for i in range(1,6):
            total_gold_blue_3_min[match_index] += participants_info[3][str(i)]['totalGold']
        for i in range(6,11):
            total_gold_red_3_min[match_index] += participants_info[3][str(i)]['totalGold']
        gold_gap[match_index] = total_gold_blue_3_min[match_index] - total_gold_red_3_min[match_index]
        
        # use df_teams
        invade_win_team[match_index] = 100 if df_teams["100"][match_index]["win"] else 200
        
        print("--------------------------------------------------")
        print("invade detected: %d" % match_index)
        print("timestamp(minutes): %.2f" % (invade_timestamp[match_index]/60000))
        print("< kills >\n  blue : %d , red : %d"% (invade_kill_blue[match_index], invade_kill_red[match_index]))
        print("< total_gold_3_min >\n  blue: %d , red: %d"% (total_gold_blue_3_min[match_index], total_gold_red_3_min[match_index]))
        print("< gold_gap >\n  %d"% gold_gap[match_index])
        print("< win >\n  %s"% ("blue" if invade_win_team[match_index] == 100 else "red"))

--------------------------------------------------
invade detected: 1
timestamp(minutes): 0.96
< kills >
  blue : 2 , red : 5
< total_gold_3_min >
  blue: 13391 , red: 13107
< gold_gap >
  284
< win >
  blue
--------------------------------------------------
invade detected: 2
timestamp(minutes): 0.70
< kills >
  blue : 7 , red : 7
< total_gold_3_min >
  blue: 14091 , red: 14424
< gold_gap >
  -333
< win >
  red
--------------------------------------------------
invade detected: 3
timestamp(minutes): 0.60
< kills >
  blue : 5 , red : 3
< total_gold_3_min >
  blue: 13675 , red: 13994
< gold_gap >
  -319
< win >
  red
--------------------------------------------------
invade detected: 5
timestamp(minutes): 1.59
< kills >
  blue : 2 , red : 4
< total_gold_3_min >
  blue: 13256 , red: 12843
< gold_gap >
  413
< win >
  red
--------------------------------------------------
invade detected: 6
timestamp(minutes): 0.59
< kills >
  blue : 4 , red : 4
< total_gold_3_min >
  blue: 13795 , red: 1

In [101]:
df_invade = pd.DataFrame([
    is_invade,
    invade_timestamp,
    invade_kill_blue,
    invade_kill_red,
    invade_win_team,
    total_gold_blue_3_min,
    total_gold_red_3_min,
    gold_gap,
    df_info_1["gameMode"]]).transpose()

df_invade.columns = [
    "is_invade",
    "invade_timestamp",
    "invade_kill_blue",
    "invade_kill_red",
    "invade_win_team",
    "total_gold_blue_3_min",
    "total_gold_red_3_min",
    "gold_gap",
    "gameMode"]

display(df_invade[(df_invade.is_invade==True) & (df_invade.invade_kill_blue > df_invade.invade_kill_red)])

# 인베가 일어났고, 블루가 레드보다 킬을 많이 딴 경기에서 블루가 이길 확률 
print("%.2f%%"%(
    len(df_invade[
        (df_invade.is_invade==True) 
        & (df_invade.invade_kill_blue > df_invade.invade_kill_red) 
        & (df_invade.invade_win_team == 100)]) 
    / len(df_invade[
        (df_invade.is_invade==True) 
        & (df_invade.invade_kill_blue > df_invade.invade_kill_red)])
    * 100)
)


,is_invade,invade_timestamp,invade_kill_blue,invade_kill_red,invade_win_team,total_gold_blue_3_min,total_gold_red_3_min,gold_gap,gameMode
3,True,36162,5,3,200,13675,13994,-319,ARAM
21,True,108428,1,0,200,4680,4235,445,CLASSIC
27,True,74669,2,0,200,4680,5155,-475,CLASSIC
34,True,55950,1,0,100,12300,11702,598,ARAM
62,True,62434,4,3,200,13746,13085,661,ARAM
64,True,51401,3,2,100,13115,12976,139,ARAM
67,True,49840,4,3,100,13023,12755,268,ARAM
68,True,76632,2,1,100,13389,13367,22,ARAM
69,True,57036,7,5,200,13035,14191,-1156,ARAM
70,True,58547,6,4,100,13766,13536,230,ARAM


36.00%


In [ ]:
match_invade

73

In [ ]:
df_invade

,is_invade,invade_timestamp,invade_kill_blue,invade_kill_red,invade_win_team,total_gold_blue_3_min,total_gold_red_3_min,gold_gap,gameMode
0,True,63063,2,2,100,7461,6960,501,URF
1,True,96234,0,1,100,7579,6743,836,URF
2,True,25730,3,7,100,8866,9835,-969,URF
3,True,44363,0,1,100,7998,7759,239,URF
4,True,98734,0,1,200,7114,7840,-726,URF
...,...,...,...,...,...,...,...,...,...
175,False,0,0,0,0,0,0,0,CLASSIC
176,False,0,0,0,0,0,0,0,CLASSIC
177,False,0,0,0,0,0,0,0,CLASSIC
178,False,0,0,0,0,0,0,0,CLASSIC
